In [1]:
import gymnasium as gym
from stk_actor.wrappers import PreprocessObservationWrapper
import torch.nn as nn
from torch.distributions import Categorical
from typing import Dict, List, Tuple, Union, Type
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
from gymnasium import spaces

def get_device(device: Union[torch.device, str] = "auto") -> torch.device:
    if device == "auto":
        device = "cuda"
    device = torch.device(device)
    if device.type == torch.device("cuda").type and not torch.cuda.is_available():
        return torch.device("cpu")
    return device

class BaseFeaturesExtractor(nn.Module):
    def __init__(self, observation_space: gym.Space, features_dim: int = 0) -> None:
        super().__init__()
        assert features_dim > 0
        self._observation_space = observation_space
        self._features_dim = features_dim
    @property
    def features_dim(self) -> int:
        return self._features_dim

def get_flattened_obs_dim(observation_space: spaces.Space) -> int:
    if isinstance(observation_space, spaces.MultiDiscrete):
        return sum(observation_space.nvec)
    else:
        return spaces.utils.flatdim(observation_space)

class FlattenExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.Space) -> None:
        super().__init__(observation_space, get_flattened_obs_dim(observation_space))
        self.flatten = nn.Flatten()
    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        return self.flatten(observations)
    
class MlpExtractor(nn.Module):
    def __init__(
        self,
        feature_dim: int,
        net_arch: Union[List[int], Dict[str, List[int]]],
        activation_fn: Type[nn.Module],
        device: Union[torch.device, str] = "auto",
    ) -> None:
        super().__init__()
        # device = torch.get_device(device)
        policy_net: List[nn.Module] = []
        value_net: List[nn.Module] = []
        last_layer_dim_pi = feature_dim
        last_layer_dim_vf = feature_dim

        if isinstance(net_arch, dict):
            pi_layers_dims = net_arch.get("pi", []) 
            vf_layers_dims = net_arch.get("vf", []) 
        else:
            pi_layers_dims = vf_layers_dims = net_arch
        for curr_layer_dim in pi_layers_dims:
            policy_net.append(nn.Linear(last_layer_dim_pi, curr_layer_dim))
            policy_net.append(activation_fn())
            last_layer_dim_pi = curr_layer_dim
        for curr_layer_dim in vf_layers_dims:
            value_net.append(nn.Linear(last_layer_dim_vf, curr_layer_dim))
            value_net.append(activation_fn())
            last_layer_dim_vf = curr_layer_dim

        self.latent_dim_pi = last_layer_dim_pi
        self.latent_dim_vf = last_layer_dim_vf
        self.policy_net = nn.Sequential(*policy_net)#.to(device)
        self.value_net = nn.Sequential(*value_net)#.to(device)

    def forward(self, features: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        :return: latent_policy, latent_value of the specified network.
            If all layers are shared, then ``latent_policy == latent_value``
        """
        return self.forward_actor(features), self.forward_critic(features)

    def forward_actor(self, features: torch.Tensor) -> torch.Tensor:
        return self.policy_net(features)

    def forward_critic(self, features: torch.Tensor) -> torch.Tensor:
        return self.value_net(features)

    
class Policy(nn.Module):
    def __init__(self, observation_space, action_dims, net_arch, activation_fn,):
        super().__init__()
        self.features_extractor = FlattenExtractor(observation_space)
        self.pi_features_extractor = self.features_extractor
        self.vf_features_extractor = self.features_extractor
        self.mlp_extractor = MlpExtractor(
            self.features_extractor.features_dim,
            net_arch=net_arch,
            activation_fn=activation_fn,
        )
        self.action_net = nn.Linear(net_arch[-1], sum(action_dims))
        self.value_net = nn.Linear(net_arch[-1], 1)


class UnifiedSACPolicy(nn.Module):
    def __init__(self, observation_space, action_dims, net_arch, activation_fn):
        super().__init__()
        
        self.shared = Policy(
            observation_space,
            action_dims,
            net_arch=net_arch,
            activation_fn=activation_fn
        )
        self.action_dims = action_dims
    
    def forward(self, x):
        x = self.shared.features_extractor(x)
        x = self.shared.mlp_extractor.policy_net(x)
        x = self.shared.action_net(x)
        return x
    
    def sample(self, x, deterministic=False):
        logits = self.forward(x)
        
        # Split logits for each action dimension
        split_logits = torch.split(logits, self.action_dims, dim=-1)
        
        actions = []
        log_probs = []
        probs = []
        
        for logit in split_logits:
            distribution = Categorical(logits=logit)
            if deterministic:
                action = torch.argmax(logit, dim=-1)
            else:
                action = distribution.sample()
            
            log_prob = distribution.log_prob(action)
            prob = F.softmax(logit, dim=-1)
            
            actions.append(action)
            log_probs.append(log_prob)
            probs.append(prob)
        
        return (
            torch.stack(actions),
            torch.stack(log_probs),
            probs
        )
    
#policy = torch.load('policy_512_512_512_512_SiLU_3_statedict', map_location='cuda')


from stable_baselines3 import PPO, A2C
from stable_baselines3.common.env_util import make_vec_env
import gymnasium as gym
from pystk2_gymnasium import AgentSpec
from bbrl.agents.gymnasium import ParallelGymAgent, make_env
from functools import partial

vec_env = make_vec_env("supertuxkart/flattened_multidiscrete-v0", seed=0, n_envs=21, wrapper_class=lambda x : (PreprocessObservationWrapper(x, ret_dict=False)), env_kwargs={
    'render_mode':None, 'agent':AgentSpec(use_ai=False, name="walid"), 'track':'minigolf', 'laps':2#'difficulty':0, #'num_kart':2, 'difficulty':0
})

tracks = ['abyss',
 'black_forest',
 'candela_city',
 'cocoa_temple',
 'cornfield_crossing',
 'fortmagma',
 'gran_paradiso_island',
 'hacienda',
 'lighthouse',
 'mines',
 'minigolf',
 'olivermath',
 'ravenbridge_mansion',
 'sandtrack',
 'scotland',
 'snowmountain',
 'snowtuxpeak',
 'stk_enterprise',
 'volcano_island',
 'xr591',
 'zengarden']
for i,venv in enumerate(vec_env.envs):
    print(i, tracks[i%len(tracks)])
    venv.env.default_track = tracks[i%len(tracks)]


net_arch=[512,512,512,512]
activation_fn=torch.nn.SiLU
filename = 'policy_512_512_512_512_SiLU_3_statedict'

action_dims = [space.n for space in vec_env.action_space]
unified_policy = UnifiedSACPolicy(
    vec_env.observation_space, 
    action_dims, 
    net_arch=net_arch, 
    activation_fn=activation_fn
)
unified_policy.load_state_dict(torch.load(filename, map_location='cpu'))


..:: Antarctica Rendering Engine 2.0 ::..
0 abyss
1 black_forest
2 candela_city
3 cocoa_temple
4 cornfield_crossing
5 fortmagma
6 gran_paradiso_island
7 hacienda
8 lighthouse
9 mines
10 minigolf
11 olivermath
12 ravenbridge_mansion
13 sandtrack
14 scotland
15 snowmountain
16 snowtuxpeak
17 stk_enterprise
18 volcano_island
19 xr591
20 zengarden


/var/folders/5b/wsyz4crx22bg_7tpsgbw3tzw0000gn/T/ipykernel_32351/3477717921.py:205: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unified_policy.load_state_dict(torch.load(f

<All keys matched successfully>

In [2]:
steps = [(
    #16384,
    3000,
    3_000_000,
)]
for n_steps, total_timesteps in steps:
    model = PPO(
        "MlpPolicy", 
        vec_env, 
        verbose=1, 
        policy_kwargs = dict(net_arch=net_arch, activation_fn=activation_fn,),
        device='cpu',
        learning_rate=0.0001,
        batch_size=128,
        n_steps=n_steps,
        tensorboard_log="./outputs/",
        ent_coef=0.001,
        clip_range=0.001,
    )
    print('DOING', n_steps, total_timesteps)
    model.policy.load_state_dict(unified_policy.shared.state_dict())
    model.policy.load_state_dict(
        PPO.load(
            "ppti_ppo_1500", 
            custom_objects={'policy_kwargs' : dict(net_arch=net_arch, activation_fn=activation_fn), }
        ).policy.state_dict()
    )
    model.learn(total_timesteps=total_timesteps, progress_bar=True)#callback=SummaryWriterCallback())
    model.save(f'ppti_ppo_{n_steps}')


Using cpu device


/opt/homebrew/Caskroom/miniforge/base/envs/bbrl/lib/python3.10/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 128, but because the `RolloutBuffer` is of size `n_steps * n_envs = 63000`, after every 492 untruncated mini-batches, there will be a truncated mini-batch of size 24
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=3000 and n_envs=21)
  warnings.warn(


..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
DOING 3000 3000000
Logging to ./outputs/PPO_5


Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 855      |
|    ep_rew_mean     | 657      |
| time/              |          |
|    fps             | 89       |
|    iterations      | 1        |
|    time_elapsed    | 705      |
|    total_timesteps | 63000    |
---------------------------------


...

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.5e+03    |
|    ep_rew_mean          | 65.4       |
| time/                   |            |
|    fps                  | 83         |
|    iterations           | 44         |
|    time_elapsed         | 33204      |
|    total_timesteps      | 2772000    |
| train/                  |            |
|    approx_kl            | 0.41051355 |
|    clip_fraction        | 0.0534     |
|    clip_range           | 0.001      |
|    entropy_loss         | -0.0116    |
|    explained_variance   | 0.972      |
|    learning_rate        | 0.0001     |
|    loss                 | 0.794      |
|    n_updates            | 430        |
|    policy_gradient_loss | 0.013      |
|    value_loss           | 8.25       |
----------------------------------------


----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.5e+03    |
|    ep_rew_mean          | 78.9       |
| time/                   |            |
|    fps                  | 83         |
|    iterations           | 45         |
|    time_elapsed         | 33951      |
|    total_timesteps      | 2835000    |
| train/                  |            |
|    approx_kl            | 0.23730262 |
|    clip_fraction        | 0.0638     |
|    clip_range           | 0.001      |
|    entropy_loss         | -0.0132    |
|    explained_variance   | 0.963      |
|    learning_rate        | 0.0001     |
|    loss                 | 1.74       |
|    n_updates            | 440        |
|    policy_gradient_loss | 0.0115     |
|    value_loss           | 10         |
----------------------------------------


----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.5e+03    |
|    ep_rew_mean          | 87.7       |
| time/                   |            |
|    fps                  | 83         |
|    iterations           | 46         |
|    time_elapsed         | 34700      |
|    total_timesteps      | 2898000    |
| train/                  |            |
|    approx_kl            | 0.51432586 |
|    clip_fraction        | 0.0752     |
|    clip_range           | 0.001      |
|    entropy_loss         | -0.0127    |
|    explained_variance   | 0.969      |
|    learning_rate        | 0.0001     |
|    loss                 | 5.98       |
|    n_updates            | 450        |
|    policy_gradient_loss | 0.0113     |
|    value_loss           | 9.7        |
----------------------------------------


----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.5e+03    |
|    ep_rew_mean          | 113        |
| time/                   |            |
|    fps                  | 83         |
|    iterations           | 47         |
|    time_elapsed         | 35448      |
|    total_timesteps      | 2961000    |
| train/                  |            |
|    approx_kl            | 0.58305323 |
|    clip_fraction        | 0.0587     |
|    clip_range           | 0.001      |
|    entropy_loss         | -0.0107    |
|    explained_variance   | 0.971      |
|    learning_rate        | 0.0001     |
|    loss                 | 5.41       |
|    n_updates            | 460        |
|    policy_gradient_loss | 0.0102     |
|    value_loss           | 10.4       |
----------------------------------------


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1.5e+03   |
|    ep_rew_mean          | 113       |
| time/                   |           |
|    fps                  | 83        |
|    iterations           | 48        |
|    time_elapsed         | 36195     |
|    total_timesteps      | 3024000   |
| train/                  |           |
|    approx_kl            | 1.0745189 |
|    clip_fraction        | 0.0591    |
|    clip_range           | 0.001     |
|    entropy_loss         | -0.012    |
|    explained_variance   | 0.968     |
|    learning_rate        | 0.0001    |
|    loss                 | 0.799     |
|    n_updates            | 470       |
|    policy_gradient_loss | 0.016     |
|    value_loss           | 11.9      |
---------------------------------------


In [3]:
model.save(f'ppti_ppo_{n_steps}')